# Formal Release

Task: On this task, you'll be creating a script using Mailchimp's API. The script must execute the following process in order:
- Create a new List.
- Add your email to the list.
- Add our emails to the list. (ryan.ramadhan@ematicsolutions.com, edwin.melendez@ematicsolutions.com, christianto@ematicsolutions.com)
- Send a campaign to the list.
- Get campaign info and print output.
Get information about a specific campaign.

Notes:
- You can use any programming language that you're comfortable with. Only creation of the Mailchimp account should be done manually. 
- You can use packages/plugins if needed.

## initialization

In [1]:
import json
import requests
import pprint

# mailchimp api_key
MAILCHIMP_API_KEY = 'f5b4a1c90813f8d3dc83bca2cbe7d57a-us18'
us_ = MAILCHIMP_API_KEY.split('-')[1]
# rearrange URL
mailchimp_api_root = "https://" + us_ + ".api.mailchimp.com/3.0/"

## Create a new List

In [2]:
def get_all_lists():
  """
  query all lists
  """
  endpoint = mailchimp_api_root + "lists/" 
  resp = requests.get(endpoint, auth=('apikey', MAILCHIMP_API_KEY)) 
  json_obj = json.loads(resp.text)
  print("Total lists:", len(json_obj['lists']) )
  #all_list.keys()
  #dict_keys(['lists', 'total_items', 'constraints', '_links'])

  #pprint.pprint(json.loads(resp.text))

  #all_list['lists'][0].keys()
  #dict_keys(['id', 'web_id', 'name', 'contact', 'permission_reminder', 'use_archive_bar', 'campaign_defaults', 'notify_on_subscribe', 'notify_on_unsubscribe', 'date_created', 'list_rating', 'email_type_option', 'subscribe_url_short', 'subscribe_url_long', 'beamer_address', 'visibility', 'double_optin', 'has_welcome', 'marketing_permissions', 'modules', 'stats', '_links'])
  for i in range(0, len(json_obj['lists'])):
      print("id = ", json_obj['lists'][i]['id'])
  return json_obj, json_obj['lists'][0]['id'] # return the first list id to do below operations

In [3]:
def add_list(new_list_name):
  """ 
  add list
  """
  endpoint = mailchimp_api_root + "lists" 
  payload = json.dumps({
              "name": new_list_name,
              "contact": 
              {
                  "company": "Test Company",
                  "address1": "Taipei",
                  "city": "Taipei",
                  "state": "Taipei",
                  "zip": "100",
                  "country": "US"
              },
              "permission_reminder": "This is only a test",
              "campaign_defaults": 
              {
                  "from_name": "Kim Lu",
                  "from_email": "kno15320@gmail.com",
                  "subject": "Test mailchimp",
                  "language": "zh-tw"
              },
              "email_type_option": True
          })

  resp = requests.post(endpoint, auth=('apikey', MAILCHIMP_API_KEY), data=payload) 
  json_obj = json.loads(resp.text)
  print(json_obj)
  return json_obj

In [4]:
all_list, list_id= get_all_lists()

Total lists: 1
id =  a40f122c8f


In [5]:
print("In below operations, we use list_id=",list_id)

In below operations, we use list_id= a40f122c8f


In [6]:
new_list=add_list("Kim new list")

{'type': 'https://mailchimp.com/developer/marketing/docs/errors/', 'title': 'User action not permitted', 'status': 403, 'detail': 'User does not have access to the requested operation', 'instance': '141f31d2-5170-4161-70d3-98847880cf95'}


## Add your email to the list.

In [7]:
def add_user_to_a_list(list_id, user_email):
  """
  subscribing user to a list
  """
  endpoint = mailchimp_api_root + "lists/" + list_id + "/members/"

  payload = json.dumps({"email_address": user_email, "status": "subscribed"})
  resp = requests.post(endpoint, auth=('apikey', MAILCHIMP_API_KEY), data=payload) 

In [8]:
def get_all_users_from_a_list(list_id):
  """
  retrieving all users from a list
  """
  endpoint = mailchimp_api_root + "lists/" + list_id + "/members/"
  resp = requests.get(endpoint, auth=('apikey', MAILCHIMP_API_KEY))

  json_obj = json.loads(resp.text)
  #print(json_obj.keys())
  #dict_keys(['members', 'list_id', 'total_items', '_links'])
  print("Totally ", len(json_obj['members']), "members")
  for i in range(0, len(json_obj['members'])):
    print(json_obj['members'][i]['email_address'])

In [ ]:
get_all_users_from_a_list(list_id)

In [23]:
# Add your email to the list.
add_user_to_a_list(list_id,"kno15320@gmail.com")

In [ ]:
get_all_users_from_a_list(list_id)

## Add our emails to the list.
(ryan.ramadhan@ematicsolutions.com, edwin.melendez@ematicsolutions.com, christianto@ematicsolutions.com)

In [25]:
for sub in ["ryan.ramadhan@ematicsolutions.com", "edwin.melendez@ematicsolutions.com", "christianto@ematicsolutions.com"]:
  add_user_to_a_list(list_id,sub)

In [26]:
get_all_users_from_a_list(list_id)

Totally  4 members
kno15320@gmail.com
ryan.ramadhan@ematicsolutions.com
edwin.melendez@ematicsolutions.com
christianto@ematicsolutions.com


#Send a campaign to the list.

In [29]:
# query all lists
def get_all_campaign():
  endpoint = mailchimp_api_root + "/campaigns" 
  resp = requests.get(endpoint, auth=('apikey', MAILCHIMP_API_KEY)) 
  json_obj = json.loads(resp.text)
  #print(type(json_obj))
  #print(json_obj.keys())
  #dict_keys(['campaigns', 'total_items', '_links'])
  
  print("Total campaigns:", len(json_obj['campaigns']) )
  #print(type(json_obj['campaigns']))#.keys())
  #<class 'list'>
  for i, cam in enumerate(json_obj['campaigns']):
    print("id = ", json_obj['campaigns'][i]['id'])
  return json_obj, json_obj['campaigns'][-1]['id'] # we use the last campaign to do below operations
    

In [30]:
def add_regular_campaign(list_id):
  endpoint = mailchimp_api_root + "campaigns/"
  # add regular campaign with a list
  payload = json.dumps({"type": "regular", "recipients":{ "list_id": list_id}, "settings": {"subject_line":"new campaign","title": "New Campaign","from_name":"Kim Lu", "reply_to":"kno15320@gmail.com"}})
  resp = requests.post(endpoint, auth=('apikey', MAILCHIMP_API_KEY), data=payload) 


In [31]:
def send_campaign(campaign_id):
  endpoint = mailchimp_api_root + "campaigns/"+campaign_id +"/actions/send"
  # add regular campaign with a list
  #payload = json.dumps({"type": "regular", "recipients":{ "list_id": MAILCHIMP_LIST_ID}})
  resp = requests.post(endpoint, auth=('apikey', MAILCHIMP_API_KEY)) 
  json_obj=json.loads(resp.text)
  return json_obj

In [ ]:
get_all_campaign()

In [14]:
add_regular_campaign(list_id)

In [32]:
_, cam_id =get_all_campaign()

Total campaigns: 2
id =  6511fe8073
id =  ae539346db


In [33]:
print("now we use campaign id", cam_id)

now we use campaign id ae539346db


In [35]:
print(send_campaign(cam_id))

{'type': 'https://mailchimp.com/developer/marketing/docs/errors/', 'title': 'Bad Request', 'status': 400, 'detail': 'Cannot send this campaign because it is currently" sent"', 'instance': 'dd16c6f4-826b-8a07-a8aa-c539912e1caa'}


## Get campaign info and print output. Get information about a specific campaign.

In [36]:
#import pprint
def print_campaign(cam_obj):
  print("===== Campaign Summary =====")
  print('\narchive_url:')
  pprint.pprint(cam_obj.get('archive_url'))

  print('\ncontent_type:')
  pprint.pprint(cam_obj.get('content_type'))

  print('\ncreate_time:')
  pprint.pprint(cam_obj.get('create_time'))

  print('\ndelivery_status:')
  pprint.pprint(cam_obj.get('delivery_status'))

  print('\nemails_sent:')
  pprint.pprint(cam_obj.get('emails_sent'))

  print('\nreport_summary')
  pprint.pprint(cam_obj.get('report_summary'))


  print('\nrecipients')
  pprint.pprint(cam_obj.get('recipients'))

In [37]:
def get_campaign_info(campaign_id):
  campaign_id= campaign_id
  endpoint = mailchimp_api_root + "/campaigns/"+ campaign_id  
  resp = requests.get(endpoint, auth=('apikey', MAILCHIMP_API_KEY)) 
  json_obj = json.loads(resp.text)
  print_campaign(json_obj)
  #json_obj.keys()
  #dict_keys(['id', 'web_id', 'type', 'create_time', 'archive_url', 'long_archive_url', 'status', 'emails_sent', 'send_time', 'content_type', 'needs_block_refresh', 'resendable', 'recipients', 'settings', 'tracking', 'report_summary', 'delivery_status', '_links'])

In [38]:
get_campaign_info(cam_id)

===== Campaign Summary =====

archive_url:
'http://eepurl.com/h6N6Ir'

content_type:
'template'

create_time:
'2022-07-16T11:22:34+00:00'

delivery_status:
{'enabled': False}

emails_sent:
4

report_summary
{'click_rate': 0,
 'clicks': 0,
 'ecommerce': {'total_orders': 0, 'total_revenue': 0, 'total_spent': 0},
 'open_rate': 0,
 'opens': 0,
 'subscriber_clicks': 0,
 'unique_opens': 0}

recipients
{'list_id': 'a40f122c8f',
 'list_is_active': True,
 'list_name': 'Kim',
 'recipient_count': 4,
 'segment_text': ''}
